# Section 3 - Cortex Analyst
### What do we know about Snowflake?
Review the stock information which is based on data you have gathered from the marketplace by running the following code below

In [ ]:
CREATE OR REPLACE VIEW DEFAULT_SCHEMA.STOCK_PRICES 

AS 

SELECT 


TICKER,
ASSET_CLASS,
PRIMARY_EXCHANGE_CODE,
PRIMARY_EXCHANGE_NAME,
DATE,
"'all-day_high'" ALL_DAY_HIGH,
"'all-day_low'" ALL_DAY_LOW,
"'nasdaq_volume'" NASDAQ_VOLUME,
"'post-market_close'" POST_MARKET_CLOSE,
"'pre-market_open'" PRE_MARKET_OPEN,
YEAR(DATE)::text YEAR,
MONTHNAME(DATE)MONTHNAME, MONTH(DATE) MONTHNO FROM (
SELECT * EXCLUDE VARIABLE_NAME
FROM FINANCE__ECONOMICS.CYBERSYN.STOCK_PRICE_TIMESERIES)


PIVOT (SUM(value) FOR VARIABLE IN (ANY ORDER BY VARIABLE));
SELECT * FROM DEFAULT_SCHEMA.STOCK_PRICES WHERE TICKER = 'SNOW'    LIMIT 5

## 3a - Analysing Market Place Shareprice Trends the traditional way
Run the cell below to see how this looks like as a Streamlit dashboard

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

from snowflake.snowpark.functions import *
from snowflake.snowpark.types import *

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()
default_ticker = 'SNOW'
st.markdown(f' #### {default_ticker} - STOCK MARKET PATTERNS')

sdate = st.number_input('Choose Year:',2020,2024,2024)
stock_table = session.table('FINANCE__ECONOMICS.CYBERSYN.STOCK_PRICE_TIMESERIES').filter(year('DATE')==sdate)

def stock_metric(variable, ticker):
    stocks = stock_table.filter((col('TICKER')==ticker) & (col('VARIABLE')==variable))
    stocks = stocks.group_by(col('DATE')).agg(min('VALUE').alias('VALUE')).order_by('DATE')
    return st.line_chart(stocks, y='VALUE',x='DATE', color = '#29B5E8')


ticker_list = stock_table.select(col('TICKER')).distinct().to_pandas()['TICKER'].tolist()
default_index = ticker_list.index(default_ticker) if default_ticker in ticker_list else 0

ticker = st.selectbox('Select Ticker:',stock_table.select(col('TICKER')).distinct().to_pandas(),default_index)

col1,col2,col3 = st.columns(3)

with col1:
    st.markdown('#### ALL DAY HIGH')
    stock_metric('all-day_high',ticker)
with col2:
    st.markdown('#### ALL DAY LOW')
    stock_metric('all-day_low',ticker)
with col3:
    st.markdown('#### NASDAQ VOLUME')
    stock_metric('nasdaq_volume',ticker)

col1,col2 = st.columns(2)
with col1:
    st.markdown('#### PRE MARKET OPEN')
    stock_metric('pre-market_open',ticker)
with col2:
    st.markdown('#### POST MARKET CLOSE')
    stock_metric('post-market_close',ticker)

st.markdown('''You will see that you can get lots of information in a dashboard - streamlit makes this capabilty very flexible.

However, some users want to be able to answer ad-hoc questions without having to create a new
dashboard or waiting for a new improved dashboard to be built.  This is where **Cortex Analyst** comes into play

Plase go to to Step 3 in the lab instructions to find out how **Cortex Analyst** works''')